# EN3160 Assignment 2 on Fitting and Alignment

Instructed by Dr. Ranga Rodrigo

Done by: Warren Jayakumar

### Import Required Libraries

In [3]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

### Detect and draw circles in the sunflower field image

In [ ]:
im = cv.imread('a2images/the_berry_farms_sunflower_field.jpeg', cv.IMREAD_REDUCED_COLOR_4)
plt.imshow(cv.cvtColor(im, cv.COLOR_BGR2RGB))
plt.axis(False)
plt.show()

In [ ]:
gray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)

# Step 3: Apply Gaussian Blur and compute Laplacian
sigma_values = range(1, 100)
log_images = []

for sigma in sigma_values:
    blurred = cv.GaussianBlur(gray, (0, 0), sigma)
    log = cv.Laplacian(blurred, cv.CV_64F)
    log_images.append((log, sigma))

# Step 4: Detect Scale-Space Extrema
extrema = []

for i in range(1, len(log_images) - 1):
    prev_log, prev_sigma = log_images[i - 1]
    curr_log, curr_sigma = log_images[i]
    next_log, next_sigma = log_images[i + 1]
    
    # Find local maxima in the scale-space
    maxima = (curr_log > prev_log) & (curr_log > next_log)
    maxima_coords = np.argwhere(maxima)
    
    for coord in maxima_coords:
        y, x = coord
        radius = curr_sigma * np.sqrt(2)
        extrema.append((x, y, radius, curr_sigma))

# Step 5: Report Parameters
extrema = sorted(extrema, key=lambda c: c[2], reverse=True)  # Sort by radius
largest_circle = extrema[0] if extrema else None

if largest_circle:
    x, y, radius, sigma = largest_circle
    print(f"Largest Circle - Center: ({x}, {y}), Radius: {radius}, Sigma: {sigma}")
    
    # Draw the largest circle on the original image
    center = (int(x), int(y))
    radius = int(radius)
    cv.circle(im, center, radius, (0, 255, 0), 2)  # Draw circle in green with thickness 2
    cv.circle(im, center, 2, (0, 0, 255), 3)  # Draw center in red with thickness 3
    
    # Display the image with the drawn circle
    plt.imshow(cv.cvtColor(im, cv.COLOR_BGR2RGB))
    plt.axis(False)
    plt.show()
else:
    print("No circles found")

print(f"Range of σ values used: {min(sigma_values)} to {max(sigma_values)}")